<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Get DataFrame from project view

**Tags:** #github #dataframe #beautifulsoup #projectview #scraping #python

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Description:** This notebook will show how to return a dataframe from project view using BeautifulSoup. It is usefull for organizations to quickly get data from GitHub project view.

**References:**
- [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [GitHub Project View](https://help.github.com/en/github/managing-your-work-on-github/about-project-boards)

## Input

### Import libraries

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

### Setup Variables
- `url`: URL of the project view page

In [83]:
url = "https://github.com/orgs/jupyter-naas/projects/10/views/19"

## Model

### Get DataFrame from project view

This function will return a dataframe from project view using BeautifulSoup.

In [48]:
# Init
data = []

# Get HTML from URL
response = requests.get(url)
html = response.text

# Parse HTML
soup = BeautifulSoup(html, "html.parser")

# Get cards
elements = soup.find_all("script", {"id": "memex-items-data"})

# Iterate over the elements and split their text
for element in elements:
    text = element.text
#     print(text)
    split_text = text.split('{"contentId":')[1:]  # Split the text as needed
    # Process the split text or perform any desired operations
#     print(split_text)
    
    for s in split_text:
        s = s.split('"memexProjectColumnId":')[1:]
        # Init
        title = None
        issue_number = None
        tmp = {
            "title": title,
            "issue_number": issue_number,
        }
        data.append(tmp)
        
        
s

['"Title","value":{"url":"https://github.com/71603764/299951448/issues/1217","state":"closed","title":{"raw":"Python - Create GitHub repository","html":"Python - Create GitHub repository"},"number":1217,"issueId":1395462430,"stateReason":null}},{',
 '"Assignees","value":[{"avatarUrl":"https://avatars.githubusercontent.com/u/72201663?s=40\\u0026u=58c93c3da53765967477e0c7a446ef5238e4a8f6\\u0026v=4","id":72201663,"login":"knshkp","url":"https://github.com/knshkp"}]},{',
 '"Status","value":{"id":"7c2c8541"}},{',
 '"Linked pull requests","value":[{"id":1076376031,"number":1220,"isDraft":false,"state":"merged","url":"https://github.com/jupyter-naas/awesome-notebooks/pull/1220"}]},{',
 '16173293,"value":{"id":"7bb90100"}},{',
 '16173294,"value":{"value":2.0}}],"content":{"id":1395462430,"url":"https://github.com/jupyter-naas/awesome-notebooks/issues/1217"}},']

In [54]:
# Init
data = []

# Get HTML from URL
response = requests.get(url)
html = response.text

# Parse HTML
soup = BeautifulSoup(html, "html.parser")

# Get cards
elements = soup.find_all("script", {"id": "memex-items-data"})

# Iterate over the elements and split their text
for element in elements:
    text = element.text
    split_text = text.split('{"contentId":')[1:]  # Split the text as needed
    
    for s in split_text:
        s = s.split('"memexProjectColumnId":')[1:]
        # Get the values using splits
        title = s[0].split('"raw":"')[-1].split('"')[0]
        issue_number = s[0].split('"number":')[-1].split(',')[0]
        assignees = s[1].split('"login":"')[-1].split('"')[0]
        status_id = s[2].split('"id":"')[-1].split('"')[0]
        PR_url = s[3].split('"url":"')[-1].split('"')[0]
        iteration_id = s[4].split('"id":"')[-1].split('"')[0]
        estimate = s[5].split('"value":')[-1].split('}')[0]
        # Create a dictionary with the values
        tmp = {
            "title": title,
            "issue_number": issue_number,
            "assignees": assignees,
            "status_id": status_id,
            "PR_url": PR_url,
            "iteration_id": iteration_id,
            "estimate": estimate,
        }
        # Append the dictionary to the data list
        data.append(tmp)

# Create a dataframe from the data list
df = pd.DataFrame(data)

# Print the dataframe
print(df)


0
[]
Empty DataFrame
Columns: []
Index: []


In [27]:
title = s[0].split('"raw":"')[-1].split('"')[0]
title

'Python - Create GitHub repository'

In [5]:
s[0]
# Number

'"Title","value":{"url":"https://github.com/71603764/299951448/issues/1217","state":"closed","title":{"raw":"Python - Create GitHub repository","html":"Python - Create GitHub repository"},"number":1217,"issueId":1395462430,"stateReason":null}},{'

In [6]:
s[1]
# Login

'"Assignees","value":[{"avatarUrl":"https://avatars.githubusercontent.com/u/72201663?s=40\\u0026u=58c93c3da53765967477e0c7a446ef5238e4a8f6\\u0026v=4","id":72201663,"login":"knshkp","url":"https://github.com/knshkp"}]},{'

In [23]:
s[0].split("raw")[-1].replace('":"', "").split('","')[0]

'Python - Create GitHub repository'

In [47]:
title = s[0].split('"raw":"')[-1].split('"')[0]
title

'Python - Create GitHub repository'

In [36]:
issue_number = s[0].split('"number":')[-1].split(',')[0]
number

'1217'

In [35]:
assignees = s[1].split('"login":"')[-1].split('"')[0]
login

'knshkp'

In [42]:
status_id = s[2].split('"id":"')[-1].split('"')[0]
ID

'7c2c8541'

In [43]:
url = s[3].split('"url":"')[-1].split('"')[0]
url

'https://github.com/jupyter-naas/awesome-notebooks/pull/1220'

In [44]:
iteration_id = s[4].split('"id":"')[-1].split('"')[0]
iteration_id

'7bb90100'

In [45]:
value = s[5].split('"value":')[-1].split('}')[0]
value

'2.0'

In [31]:
s[1]
# Login

'"Assignees","value":[{"avatarUrl":"https://avatars.githubusercontent.com/u/72201663?s=40\\u0026u=58c93c3da53765967477e0c7a446ef5238e4a8f6\\u0026v=4","id":72201663,"login":"knshkp","url":"https://github.com/knshkp"}]},{'

In [37]:
s[2]
# ID status

'"Status","value":{"id":"7c2c8541"}},{'

In [38]:
s[3]
# linked PR number

'"Linked pull requests","value":[{"id":1076376031,"number":1220,"isDraft":false,"state":"merged","url":"https://github.com/jupyter-naas/awesome-notebooks/pull/1220"}]},{'

In [39]:
s[4]
# ID iteration

'16173293,"value":{"id":"7bb90100"}},{'

In [40]:
s[5]
# Value

'16173294,"value":{"value":2.0}}],"content":{"id":1395462430,"url":"https://github.com/jupyter-naas/awesome-notebooks/issues/1217"}},'

## Output

### Display result

In [9]:
df = pd.DataFrame(data)
df

,title,issue_number
0,None,None
